### Problem Statement 
Banks process millions of transactions daily, with critical needs to understand customer behavior, transaction patterns, and detect fraudulent activity early. This project aims to develop a Banking Transaction Insights system to:
- Analyze customer demographics linked to bank accounts and transaction activity.


- Identify transaction trends across account types, branches, and regions.


- Detect anomalies and potentially fraudulent transactions.


- Provide an interactive Streamlit app to explore transaction data with filters and CRUD operations on accounts and transactions.


### Business Use Cases
- Profiling customer transaction behavior to tailor banking products.


- Identifying high-risk transactions and accounts for fraud prevention.


- Branch performance evaluation based on transactions and accounts.


- Enabling customers and bank officials to query transaction histories interactively.


## Approach
### Data Preparation
- Load and clean the datasets: customers, accounts, and transactions, etc.


- Ensure data consistency, remove duplicates, and validate foreign key relationships.


In [1]:
# Libraries & Setup
import pandas as pd

**Dataset:** 

1. Customers Dataset: [customers.csv](https://drive.google.com/file/d/13DJDriNwpafoe-RJNDJsOLg0PVIW-e8E/view?usp=sharing) 
2. Account Balances Dataset: [accounts.csv](https://drive.google.com/file/d/1mZgRcOhtPyvZV-uZAXFFg_8ObOnlyAR_/view?usp=sharing) 
3. Transactions Dataset: [transactions.csv](https://drive.google.com/file/d/1oiJBpB8CYs9cMf9Vnk3tuelfgG-X57we/view?usp=sharing)
4. Loans Dataset: [loans.csv](https://drive.google.com/file/d/1-kUiGCcjcz7OgUSLPWf2-0KTh_Nriftd/view?usp=sharing) / [loans.json](https://drive.google.com/file/d/19rY69Q40oletabujXQUTIkvLUJs7E3xc/view?usp=sharing)
5. Credit Cards Dataset: [credit_cards.json](https://drive.google.com/file/d/1EZ4fQ1wkXHU1niMx7J9vU5vc1RhTfbLi/view?usp=sharing)
6. Branches Dataset: [branches.csv](https://drive.google.com/file/d/1704SFQek5mrEDDa_8GphDuw8IcTUYKdO/view?usp=sharing) / [branches.json](https://drive.google.com/file/d/13Vy8NDQTMg_7X0tLj0FVyXSSQWtu5nHk/view?usp=sharing)
7. Support Tickets Dataset: [support_tickets.csv](https://drive.google.com/file/d/1MLRTxA4SSNpKIvGd2Cytzr6CFF-3JiZM/view?usp=sharing) / [support_tickets.json](https://drive.google.com/file/d/17jRVGXvbU-RBEQJgQVCCKu_XxP5au1YH/view?usp=sharing)

### STEP 1 - ALL DATASETS LOADED

In [2]:
# STEP 1 – all Datasets loaded

customers = pd.read_csv("customers.csv")
accounts = pd.read_csv("accounts.csv")
transactions = pd.read_csv("transactions.csv")

loans = pd.read_csv("loans.csv")          # ya pd.read_json("loans.json")
credit = pd.read_json("credit_cards.json")

branches = pd.read_csv("branches.csv")    # ya json
tickets = pd.read_csv("support_tickets.csv")

### STEP 2 – CUSTOMERS CLEANING (MASTER TABLE)

### 1. Customers Dataset:
Contains core demographic details and primary account type for bank customers.
- customer_id (Alphanumeric String) – Unique identifier for each bank customer.
- name (String) – The full name of the customer.
- gender (String) – The customer's gender (e.g., M, F).
- age (Integer) – The age of the customer.
- city (String) – The city of residence for the customer.
- account_type (String) – The type of the customer's primary account (e.g., Savings, Current).
- join_date (Date) – The date the customer joined the bank.

In [3]:
# Duplicate remove
customers.drop_duplicates(subset="customer_id", inplace=True)

# Age valid
customers = customers[(customers["age"] > 0) & (customers["age"] < 100)]

# Gender normalize
customers["gender"] = customers["gender"].str.upper().map({
    "MALE": "M", "FEMALE": "F", "M": "M", "F": "F"
})

# Date fix
customers["join_date"] = pd.to_datetime(customers["join_date"], errors="coerce")

# Save
customers.to_csv("customers_clean.csv", index=False)
print("Customers cleaned and saved to customers_clean.csv")
print(customers.head())


Customers cleaned and saved to customers_clean.csv
  customer_id              name gender  age            city account_type  \
0       C0001    Richard Wilson      M   65   Kimberlyburgh      Savings   
1       C0002      Holly Parker      F   41      Port David      Current   
2       C0003     Megan Alvarez      M   46  West Laurieton      Savings   
3       C0004  Nicholas Stewart      F   39        Amyville      Current   
4       C0005  Douglas Anderson      M   70   New Craigport      Current   

   join_date  
0 2023-08-27  
1 2023-12-08  
2 2022-05-25  
3 2020-11-30  
4 2023-01-29  


### STEP 3 – ACCOUNTS CLEANING

### 2. Account Balances Dataset:
Contains the latest account balance information linked to each customer.
- customer_id (Alphanumeric String) – Linked customer identifier.
- account_balance (Float) – The current total balance held by the customer.
- last_updated (DateTime) – The date and time when the account balance was last updated.


In [4]:
# Duplicate
accounts.drop_duplicates(inplace=True)

# Date
accounts["last_updated"] = pd.to_datetime(accounts["last_updated"], errors="coerce")

# Customer ID match
accounts = accounts[
    accounts["customer_id"].isin(customers["customer_id"])
]

accounts.to_csv("accounts_clean.csv", index=False)
print("Accounts cleaned and saved to accounts_clean.csv")
print(accounts.head())

Accounts cleaned and saved to accounts_clean.csv
  customer_id  account_balance        last_updated
0       C0001        293729.87 2025-05-24 09:46:29
1       C0002        150735.15 2025-05-24 09:46:29
2       C0003        170156.21 2025-05-24 09:46:29
3       C0004        311383.96 2025-05-24 09:46:29
4       C0005         60320.75 2025-05-24 09:46:29


### STEP 4 – TRANSACTIONS CLEANING

### 3. Transactions Dataset:
Contains a detailed log of all transactions conducted by customers.
- txn_id (Alphanumeric String) – Unique identifier for each transaction.
- customer_id (Alphanumeric String) – Linked customer identifier who performed the transaction.
- txn_type (String) – The nature of the transaction (e.g., deposit, withdrawal, transfer, online fraud, purchase).
- amount (Float) – The monetary amount involved in the transaction.
- txn_time (DateTime) – The exact date and time the transaction occurred.
- status (String) – The final outcome of the transaction (e.g., success, failed).


In [5]:
transactions.drop_duplicates(subset="txn_id", inplace=True)

transactions = transactions[transactions["amount"] > 0]

transactions["txn_time"] = pd.to_datetime(
    transactions["txn_time"], errors="coerce"
)

transactions = transactions.dropna(subset=["txn_time"])

transactions = transactions[
    transactions["customer_id"].isin(customers["customer_id"])
]

transactions.to_csv("transactions_clean.csv", index=False)
print("Transactions cleaned and saved to transactions_clean.csv")
print(transactions.head())

Transactions cleaned and saved to transactions_clean.csv
   txn_id customer_id    txn_type    amount            txn_time   status
0  T00001       C0363     deposit  44442.71 2025-02-14 06:26:00  success
1  T00002       C0432     deposit  15737.59 2025-03-27 07:21:21  success
2  T00003       C0084  withdrawal  82150.70 2024-07-31 18:28:06  success
3  T00004       C0067  withdrawal  20313.86 2024-11-11 12:18:47  success
4  T00005       C0485  withdrawal  74536.08 2025-04-15 10:52:59  success


### STEP 5 – LOANS CLEANING (IMPORTANT ID NORMALIZATION)

### 4. Loans Dataset:
Contains details for various types of loans issued by the bank.
- Loan_ID (Integer) – Unique identifier for each loan.
- Customer_ID (Integer) – Linked customer identifier.
- Account_ID (Integer) – Linked account identifier.
- Branch (String) – The branch responsible for managing the loan.
- Loan_Type (String) – The category of the loan (e.g., Personal, Business, Home, Auto).
- Loan_Amount (Integer) – The principal amount of the loan.
- Interest_Rate (Float) – The annual interest rate applied to the loan.
- Loan_Term_Months (Integer) – The duration of the loan agreement, in months.
- Start_Date (Date) – The date the loan was initiated.
- End_Date (Date) – The scheduled date for the loan to be fully repaid.
- Loan_Status (String) – The current operational status of the loan (e.g., Active, Closed, Approved).


In [6]:
import pandas as pd

loans = pd.read_csv("loans.csv")

# Remove duplicates
loans.drop_duplicates(subset="Loan_ID", inplace=True)

# Date cleaning
loans["Start_Date"] = pd.to_datetime(loans["Start_Date"], errors="coerce")
loans["End_Date"]   = pd.to_datetime(loans["End_Date"], errors="coerce")
loans = loans[loans["Start_Date"] < loans["End_Date"]]

# Loan status validation
valid_status = ["Active", "Closed", "Approved", "Defaulted"]
loans = loans[loans["Loan_Status"].isin(valid_status)]

# Save clean loans
loans.to_csv("loans_clean.csv", index=False)
print("Loans cleaned and saved to loans_clean.csv")

print("Loans cleaned rows:", loans.shape)


Loans cleaned and saved to loans_clean.csv
Loans cleaned rows: (553, 11)


### STEP 6 - CREDIT CARD CLEANING

### 5. Credit Cards Dataset:
Contains details for all credit cards associated with bank accounts.
- Card_ID (Integer) – Unique identifier for each credit card.
- Customer_ID (Integer) – Linked customer identifier.
- Account_ID (Integer) – Linked account identifier.
- Branch (String) – The branch managing the credit card.
- Card_Number (String) – The credit card number (as a string).
- Card_Type (String) – The tier or type of the card (e.g., Business, Platinum, Gold, Silver).
- Card_Network (String) – The payment network (e.g., Visa, RuPay, Amex, MasterCard).
- Credit_Limit (Integer) – The maximum spending limit on the card.
- Current_Balance (Float) – The current outstanding balance/debt on the card.
- Issued_Date (Date) – The date the card was issued.
- Expiry_Date (Date) – The date the card is set to expire.
- Status (String) – The current operational status of the card (e.g., Active, Expired, Blocked).


In [7]:
# Date cleaning
credit["Issued_Date"] = pd.to_datetime(credit["Issued_Date"], errors="coerce")
credit["Expiry_Date"] = pd.to_datetime(credit["Expiry_Date"], errors="coerce")

credit = credit[credit["Issued_Date"] < credit["Expiry_Date"]]

# Status validation
valid_status = ["Active", "Expired", "Blocked"]
credit = credit[credit["Status"].isin(valid_status)]

# Save clean file
credit.to_csv("credit_cards_clean.csv", index=False)
print("Credit cards cleaned:", credit.shape)
print("Credit cards cleaned and saved to credit_cards_clean.csv")

Credit cards cleaned: (557, 12)
Credit cards cleaned and saved to credit_cards_clean.csv


### STEP 7 - BRANCHES CLEANING

### 6. Branches Dataset:
Contains detailed information about each of the bank's branches.
- Branch_ID (Integer) – Unique identifier for the bank branch.
- Branch_Name (String) – The full name of the branch.
- City (String) – The city where the branch is physically located.
- Manager_Name (String) – The name of the manager overseeing the branch.
- Total_Employees (Integer) – The number of employees working at the branch.
- Branch_Revenue (Float) – The total revenue generated by the branch.
- Opening_Date (Date) – The date the branch commenced operations.
- Performance_Rating (Integer) – A rating (1-5) indicating the branch's performance.

In [8]:
# Date fix
branches["Opening_Date"] = pd.to_datetime(
    branches["Opening_Date"], errors="coerce"
)

# Rating check
branches = branches[
    (branches["Performance_Rating"] >= 1) &
    (branches["Performance_Rating"] <= 5)
]

# Save clean branches
branches.to_csv("branches_clean.csv", index=False)
print("Branches cleaned:", branches.shape)
print("Branches cleaned and saved to branches_clean.csv")

Branches cleaned: (520, 8)
Branches cleaned and saved to branches_clean.csv


### STEP 8 - SUPPORT TICKETS CLEANING

### 7. Support Tickets Dataset:
Contains records of all customer service interactions, including the issue details, resolution status, and customer feedback.
- Ticket_ID (Alphanumeric String) – Unique identifier for each support ticket.
- Customer_ID (Alphanumeric String) – Linked customer identifier.
- Account_ID (Alphanumeric String) – Linked account identifier.
- Loan_ID (Integer/String) – Linked loan identifier (may be empty if the ticket is not loan-related).
- Branch_Name (String) – The bank branch associated with the ticket or customer.
- Issue_Category (String) – The type of problem reported (e.g., Loan Payment Delay, Card Not Working, EMI Auto-debit Failed).
- Description (String) – A short summary of the issue raised by the client.
- Date_Opened (Date) – The date the support ticket was created.
- Date_Closed (Date) – The date the ticket was resolved or closed.
- Priority (String) – The urgency level of the ticket (e.g., Critical, Low).
- Status (String) – The current state of the ticket (e.g., Resolved, Closed, In Progress).
- Resolution_Remarks (String) – Detailed notes on the solution provided.
- Support_Agent (String) – The name of the agent who handled the ticket.
- Channel (String) – The method of communication (e.g., Email, Phone, In-person).
- Customer_Rating (Integer) – The customer's satisfaction rating for the support service (e.g., 1-5).


In [16]:
import pandas as pd

# =========================
# STEP 1: Load raw dataset
# =========================
tickets = pd.read_csv("support_tickets.csv")

# =========================
# STEP 2: Convert date columns to datetime
# =========================
tickets["Date_Opened"] = pd.to_datetime(
    tickets["Date_Opened"], errors="coerce"
)

tickets["Date_Closed"] = pd.to_datetime(
    tickets["Date_Closed"], errors="coerce"
)

# =========================
# STEP 3: Remove invalid closed / resolved tickets
# Rule:
# Closed / Resolved tickets MUST have Date_Closed
# =========================
tickets = tickets[
    ~(
        tickets["Status"].isin(["Closed", "Resolved"]) &
        tickets["Date_Closed"].isna()
    )
]

# =========================
# STEP 4: Calculate Resolution_Days
# =========================
tickets["Resolution_Days"] = (
    tickets["Date_Closed"] - tickets["Date_Opened"]
).dt.days

# =========================
# STEP 5: Handle invalid Resolution_Days
# - Negative days → remove
# - NaN allowed for open / in-progress tickets
# =========================
tickets = tickets[
    tickets["Resolution_Days"].isna() |
    (tickets["Resolution_Days"] >= 0)
]

# =========================
# STEP 6: Handle Loan_ID missing values
# Float → Nullable Integer (SQL safe)
# =========================
tickets["Loan_ID"] = (
    pd.to_numeric(tickets["Loan_ID"], errors="coerce")
    .astype("Int64")
)

# =========================
# STEP 7: Handle Resolution_Days datatype
# =========================
tickets["Resolution_Days"] = tickets["Resolution_Days"].astype("Int64")

# =========================
# STEP 8: Handle OTHER missing values (IMPORTANT)
# =========================

# Text columns → fill with meaningful defaults
text_fill_cols = [
    "Resolution_Remarks",
    "Description"
]

for col in text_fill_cols:
    tickets[col] = tickets[col].fillna("Not Provided")

# Priority missing (rare case)
tickets["Priority"] = tickets["Priority"].fillna("Unknown")

# Channel missing
tickets["Channel"] = tickets["Channel"].fillna("Unknown")

# Customer_Rating safety (if ever missing)
tickets["Customer_Rating"] = tickets["Customer_Rating"].fillna(0).astype(int)

# =========================
# STEP 9: Final validation check
# =========================
print(tickets.info())
print("Final rows after cleaning:", tickets.shape[0])

# =========================
# STEP 10: Save SQL-ready clean file
# =========================
tickets.to_csv("support_tickets_clean.csv", index=False)


<class 'pandas.core.frame.DataFrame'>
Index: 556 entries, 0 to 599
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Ticket_ID           556 non-null    object        
 1   Customer_ID         556 non-null    object        
 2   Account_ID          556 non-null    object        
 3   Loan_ID             213 non-null    Int64         
 4   Branch_Name         556 non-null    object        
 5   Issue_Category      556 non-null    object        
 6   Description         556 non-null    object        
 7   Date_Opened         556 non-null    datetime64[ns]
 8   Date_Closed         496 non-null    datetime64[ns]
 9   Priority            556 non-null    object        
 10  Status              556 non-null    object        
 11  Resolution_Remarks  556 non-null    object        
 12  Support_Agent       556 non-null    object        
 13  Channel             556 non-null    object        
 14 